In [1]:
# author=cxf
# date=2020-8-8
# file for test predicted cutoff

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as mp 
import warnings
warnings.filterwarnings("ignore")
index1=['sample']
index2=[i for i in range(0,11)]
index_list=index1+index2

# get result predicted
test_x=pd.read_csv('../3.model_training/test_feature.csv',index_col=0)
test_y=pd.read_csv('../3.model_training/test_cutoff.csv',index_col=0)

# merge sites information


# merge error rate
error_run1=pd.read_csv('../0.prepare_processing/run1/error.txt',names=index_list,index_col=0)
error_run2=pd.read_csv('../0.prepare_processing/run2/error.txt',names=index_list,index_col=0)
error_all=pd.concat([error_run1,error_run2],axis=0,sort=False)

In [2]:
import math
from  scipy.stats import ttest_rel
precision_dict={90:[0.02,0.05],95:[0.01,0.025],99:[0.002,0.005]}
for i in range(1,6): 
    for precision,error_rate in precision_dict.items():
        # merge sites information
        df_sites_run1=pd.read_csv(f'../0.prepare_processing/run1/a{precision}.txt',names=index_list,index_col=0)
        df_sites_run2=pd.read_csv(f'../0.prepare_processing/run2/a{precision}.txt',names=index_list,index_col=0)
        df_sites_all=pd.concat([df_sites_run1,df_sites_run2],axis=0,sort=False)
    # error rate and number of genotyped sites model vs different stiff cutoff with different criterion
        df_x=test_x[test_x['precise']==precision]
        df_y=pd.DataFrame(test_y[test_x['precise']==precision]['pred_cutoff'])
        df_error=pd.merge(df_y,error_all,on='sample')
        df_sites=pd.merge(df_y,df_sites_all,on='sample')
        for error in error_rate:
            # number of samples passed at different cutoffs
            cutoff_pass_num=df_error[df_error[i]<error].shape[0]
            # number of sites genotyped of samples passed at different cutoffs
            cutoff_pass_sites=df_sites[df_error[i]<error]
            # site and sample number at predicted cutoff 
            total=df_error.shape[0]
            predict_pass_num=0
            predict_pass_sites=[]
            predict_error_rate=[]
            for item in df_error.iterrows():
                cutoff=item[1]['pred_cutoff']
                error_rate=item[1][int(cutoff)]
                predict_error_rate.append(error_rate)
                if error_rate<error:
                    predict_pass_num+=1

            for item in df_sites.iterrows():
                cutoff=item[1]['pred_cutoff']
                sites=item[1][int(cutoff)]
                predict_pass_sites.append(sites)

            # paired T test
            df_x['sites']=predict_pass_sites
            df_x['error']=predict_error_rate
            df_predict_pass_sites=df_x[df_x['error']<error]
            total_num=df_y.shape[0]

            if cutoff_pass_sites.shape[0]>1:
                X_sum=cutoff_pass_sites.shape[0]
                df_predict_and_real=pd.merge(df_predict_pass_sites,cutoff_pass_sites,on='sample')
                print(f"{i}X,{precision},{error},{df_predict_pass_sites.shape[0]}({round(df_predict_pass_sites.shape[0]/total_num*100,2)}%),{X_sum}({round(X_sum/total_num*100,2)}%),{round(df_predict_and_real['sites'].mean(),2)}±{round(df_predict_and_real['sites'].std()/math.sqrt(df_predict_and_real.shape[0]),2)},{round(df_predict_and_real[i].mean(),2)}±{round(df_predict_and_real[i].std()/math.sqrt(df_predict_and_real.shape[0]),2)},{round(ttest_rel(df_predict_and_real[i], df_predict_and_real['sites'])[0],2)},{'%.2e'%ttest_rel(df_predict_and_real[i], df_predict_and_real['sites'])[1]}")
            else:
                X_sum=0
                df_predict_and_real=df_predict_pass_sites
                print(f"{i}X,{precision},{error},{df_predict_pass_sites.shape[0]}({round(df_predict_pass_sites.shape[0]/total_num*100,2)}%),0,{round(df_predict_and_real['sites'].mean(),2)}±{round(df_predict_and_real['sites'].std()/math.sqrt(df_predict_and_real.shape[0]),2)},0,-,-")
            #if X_sum>1:    
                #print(f"{i}X,{precision},{error},{df_predict_pass_sites.shape[0]}({round(df_predict_pass_sites.shape[0]/total_num*100,2)}%),{X_sum}({round(X_sum/total_num*100,2)}%),{round(df_predict_and_real['sites'].mean(),2)}±{round(df_predict_and_real['sites'].std()/math.sqrt(df_predict_and_real.shape[0]),2)},{round(df_predict_and_real[0].mean(),2)}±{round(df_predict_and_real[0].std()/math.sqrt(df_predict_and_real.shape[0]),2)},{round(ttest_rel(df_predict_and_real[0], df_predict_and_real['sites'])[0],2)},{'%.2e'%ttest_rel(df_predict_and_real[0], df_predict_and_real['sites'])[1]}")
            #else:
                #print(f"{i}X,{precision},{error},{df_predict_pass_sites.shape[0]}({round(df_predict_pass_sites.shape[0]/total_num*100,2)}%),0,{round(df_predict_and_real['sites'].mean(),2)}±{round(df_predict_and_real['sites'].std()/math.sqrt(df_predict_and_real.shape[0]),2)},0,-,-")

1X,90,0.02,264(100.0%),239(90.53%),942.2±12.8,943.14±12.68,2.55,1.14e-02
1X,90,0.05,264(100.0%),259(98.11%),903.59±14.79,905.8±14.54,4.45,1.27e-05
1X,95,0.01,256(96.97%),200(75.76%),806.77±13.4,806.94±13.35,0.97,3.33e-01
1X,95,0.025,264(100.0%),242(91.67%),740.68±15.8,741.54±15.69,3.14,1.92e-03
1X,99,0.002,223(84.47%),69(26.14%),443.84±17.39,445.09±17.28,1.3,1.99e-01
1X,99,0.005,261(98.86%),154(58.33%),320.92±14.92,321.16±14.73,0.3,7.62e-01
2X,90,0.02,264(100.0%),264(100.0%),891.91±15.52,883.13±15.75,-11.34,1.55e-24
2X,90,0.05,264(100.0%),264(100.0%),891.91±15.52,883.13±15.75,-11.34,1.55e-24
2X,95,0.01,256(96.97%),263(99.62%),701.45±17.7,691.35±17.71,-11.09,1.41e-23
2X,95,0.025,264(100.0%),264(100.0%),691.19±17.72,681.16±17.74,-11.31,1.95e-24
2X,99,0.002,223(84.47%),213(80.68%),266.15±13.33,260.81±12.99,-6.73,1.75e-10
2X,99,0.005,261(98.86%),255(96.59%),227.84±12.22,223.29±11.94,-6.79,7.98e-11
3X,90,0.02,264(100.0%),264(100.0%),891.91±15.52,872.36±16.16,-12.42,3.62e-28
3X,90,0.05,264(1